In [1]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

In [12]:
def PercolationCheck(Matrix,L,i,j,i2,j2,rooti,rootj):
    '''
    Checks if adding the site to the grid made a cluster percolate or not.
    
    That means it calculates all the possible distances(wrapping or not) to the root node for two different
    sites that belong to the same cluster, finds the smallest distance among them and see if it percolates or
    not, which is when the distances of these two sites differ by a factor different than 1.
    returns:
        True/False    
    '''
    

    Dist_x=np.zeros((2,3))
    Dist_y=np.zeros((2,3))
    
    #Normal Distance (dy):
    Dist_y[0,0]=abs(i-rooti)
    #Wraping upwards:
    Dist_y[0,1]=abs((L-1-rooti)+i)
    #Wraping downwards:
    Dist_y[0,2]=abs((L-1-i)+rooti)
    
    #Normal Distance (dy):
    Dist_y[1,0]=abs(i2-rooti)
    #Wraping upwards:
    Dist_y[1,1]=abs((L-1-rooti)+i2)
    #Wraping downwards:
    Dist_y[1,2]=abs((L-1-i2)+rooti)
    
    #Normal Distance (dx):
    Dist_x[0,0]=abs(j-rootj)
    #Wraping through the right:
    Dist_x[0,1]=abs((L-1-j)+rootj)
    #Wraping through the left:
    Dist_x[0,2]=abs((L-1-rooti)+j)
    
    #Normal Distance (dx):
    Dist_x[1,0]=abs(j2-rootj)
    #Wraping through the right:
    Dist_x[1,1]=abs((L-1-j2)+rootj)
    #Wraping through the left:
    Dist_x[1,2]=abs((L-1-rooti)+j2)
    
    minimal_distance_x=np.zeros(2)
    minimal_distance_y=np.zeros(2)
    
    minimal_distance_x=np.min(Dist_x[0,:]),np.min(Dist_x[1,:])
    minimal_distance_y=np.min(Dist_y[0,:]),np.min(Dist_y[1,:])
    
    Distance=np.zeros(2)
    
    Distance[0]=math.sqrt(minimal_distance_x[0]**2+minimal_distance_y[0]**2)
    Distance[1]=math.sqrt(minimal_distance_x[1]**2+minimal_distance_y[1]**2)
    
    if (abs(Distance[0]-Distance[1])>1.0):
        return(True)
    else:
        return(False)
    
            
    

In [13]:
def Clustering(Matrix,L,i,j,Pointers,Connections):
    '''
    Adds the site (i,j) to the grid and does the clustering, re-labels and alters the pointers when needed.
    returns:
        Matrix: Matrix with the labels for each cluster
        Pointers: Stores the pointers for each site's cluster/root
        FinalState: if it percolates or not
    '''
    
    
    N=L*L
    #Check sizes:
    Sizes=np.zeros(4,dtype=int)
    Root=np.zeros((4,2),dtype=int)
    FinalState='Doesnt Percolate!'
    neighbours=0
    #Checks which neighour belongs to the largest cluster
    for n in range(4):
        if(Connections[n,2]!=0):
            i2,j2=Connections[n,0],Connections[n,1]
            if(Pointers[i2*L+j2,0]!=L):
                Root[n,:]=Pointers[i2*L+j2,:]
                for k in range(N):
                    if(Pointers[k,0]==Root[n,0] and Pointers[k,1]==Root[n,1]):
                        Sizes[n]+=1            
            else:
                Root[n,:]=i2,j2
                Sizes[n]=1
                
            neighbours+=1
    
    #Connects the site to biggest cluster
    Matrix[i,j]=Connections[np.argmax(Sizes),2]
    Pointers[i*L+j,0]=Root[np.argmax(Sizes),0]
    Pointers[i*L+j,1]=Root[np.argmax(Sizes),1]    
    
    
    #Reconnects and relabels the rest if needed
    if(neighbours>1):
        for n in range(4):
            if(Connections[n,2]!=0 and n!=np.argmax(Sizes)):
                i2,j2=Connections[n,0],Connections[n,1]
                rooti,rootj=Pointers[i2*L+j2,0],Pointers[i2*L+j2,1]
                #means it's a single site that doesnt belong to any cluster:
                if(rooti==L):                        
                    Pointers[i2*L+j2,0],Pointers[i2*L+j2,1]=Root[np.argmax(Sizes),0],Root[np.argmax(Sizes),1]
                    Matrix[i2,j2]=Connections[np.argmax(Sizes),2]
                    for k in range(N):
                        if(Pointers[k,0]==i2 and Pointers[k,1]==j2):
                            Pointers[k,0],Pointers[k,1]=Root[np.argmax(Sizes),0],Root[np.argmax(Sizes),1]
                            Matrix[int(k/L),int(k%L)]=Connections[np.argmax(Sizes),2]
                else:
                    #if it has the same label as the other cluster nothing changes. Checks for percolation
                    if(Matrix[i2,j2]==Connections[np.argmax(Sizes),2]):
                        state=PercolationCheck(Matrix,L,i,j,i2,j2,rooti,rootj)
                        if(state):
                            FinalState='Percolates!'
                    #if not re-labels and alters all the pointers of that cluster
                    else:  
                        for k in range(N):
                            if(Pointers[k,0]==rooti and Pointers[k,1]==rootj):
                                Pointers[k,0],Pointers[k,1]=Root[np.argmax(Sizes),0],Root[np.argmax(Sizes),1]
                                Matrix[int(k/L),int(k%L)]=Connections[np.argmax(Sizes),2]
                            elif(Pointers[k,0]==i2 and Pointers[k,1]==j2):
                                Pointers[k,0],Pointers[k,1]=Root[np.argmax(Sizes),0],Root[np.argmax(Sizes),1]
                                Matrix[int(k/L),int(k%L)]=Connections[np.argmax(Sizes),2]

                        Matrix[rooti,rootj]=Connections[np.argmax(Sizes),2]
                        Pointers[rooti*L+rootj,0],Pointers[rooti*L+rootj,1]=Root[np.argmax(Sizes),0],Root[np.argmax(Sizes),1]
                            
    return(Matrix,Pointers,FinalState)
        
        

In [14]:
def PercolationAlgorithm(Matrix,L,i,j,Pointers,label):
    '''
    Looks at the neighbours of site (i,j) in the grid and does the clustering
    returns:
        Matrix: Matrix with the labels for each cluster
        Pointers: Stores the pointers for each site's cluster/root
        label: the next label to be used
        finalstate: if it percolates or not
    '''
    
    Connections=np.zeros((4,3),dtype=int)
    finalstate='Doesnt Percolate!'
    
    if(i+1==L):
        downi=0
    else:
        downi=i+1
    if(i-1<0):
        upi=L-1
    else:
        upi=i-1
        
    if(j+1==L):
        rightj=0
    else:
        rightj=j+1
    if(j-1<0):
        leftj=L-1
    else:
        leftj=j-1
    
    #Connections: Stores the indexes of the neighbours and it's label
    
    root=True
    if(Matrix[i,rightj]!=0):
        Connections[0,:]=i,rightj,Matrix[i,rightj]
        root=False
    if(Matrix[i,leftj]!=0):
        Connections[1,:]=i,leftj,Matrix[i,leftj]
        root=False
    if(Matrix[upi,j]!=0):
        Connections[2,:]=upi,j,Matrix[upi,j]
        root=False
    if(Matrix[downi,j]!=0):
        Connections[3,:]=downi,j,Matrix[downi,j]
        root=False
    
    if(root):
        Matrix[i,j]=label
        label=label+1
    else:
        Matrix,Pointers,finalstate=Clustering(Matrix,L,i,j,Pointers,Connections)
    
    return(Matrix,Pointers,label,finalstate)

In [15]:
def filtering(arr,x,y):
    if arr[0]!=x and arr[1]!=y:
        return(True)
    else:
        return(False)

In [16]:
def DataCleaner(PercolatingClusters,Pointers,state,L,i,j):
    '''
    Keeps track of all the percolating clusters (infinite clusters) and removes it from the count;
    returns:
        PercolatingClusters: An array that contains every percolating cluster
        clusters: An array with the cleaned data (without the infinite clusters)
        clusters_inf: An Array with every cluster (including the infinite clusters)
    '''
    
    if(state=='Percolates!'):
        rooti,rootj=Pointers[i*L+j,:]
        x=(rooti,rootj)
        if x not in PercolatingClusters:        
            PercolatingClusters=np.append(PercolatingClusters,[[rooti,rootj]],axis=0)
        new = Pointers
        for k in range(len(PercolatingClusters[:,0])):
            rooti,rootj=PercolatingClusters[k,:]
            new = np.array([x for x in new if filtering(x,rooti,rootj)])

        clusters=np.unique(new,return_counts=True,axis=0)
        clusters=np.unique(clusters[1],return_counts=True)
    else:
        new=Pointers
        for k in range(len(PercolatingClusters[:,0])):
            rooti,rootj=PercolatingClusters[k,:]
            new = np.array([x for x in new if filtering(x,rooti,rootj)])

        clusters=np.unique(new,return_counts=True,axis=0)
        clusters=np.unique(clusters[1],return_counts=True)
        
        
    clusters_inf=np.unique(Pointers,return_counts=True,axis=0)
    clusters_inf=np.unique(clusters_inf[1],return_counts=True)   
        
        
    return(PercolatingClusters,clusters,clusters_inf)

In [21]:
def main(L,Avg):
    N=L*L
    steps=int(L/10)
    count=0
    f=open('result500_L40_SecondAnalysis.dat','w')
    print('#L=',L,'Avg:',Avg,'\n#n X U P_infCluster P2_infCluster P4_infCluster Vol_inf',file=f)
    
    results=np.zeros((N,6))
    for simn in range(Avg):
        Pointers=np.ones([N,2],dtype=int)*L
        Matrix=np.zeros((L,L),dtype=int)
        PercolatingClusters=np.zeros((0,2),dtype=int)
        #Arrays that contains the indexes (coordinates) of the root node of every cell; 
        #(L,L): means a cell that does not belong to any cluster
        
        #starting label for the clusters
        label=1    
        for n in range(1,N-L,steps):
            for step in range(steps):
                while(True):
                    i=int(rnd.uniform(0,L))
                    j=int(rnd.uniform(0,L))
                    if(Matrix[i,j]==0):
                        break
                Matrix,Pointers,label,state=PercolationAlgorithm(Matrix,L,i,j,Pointers,label)
                PercolatingClusters,clusters,clusters_inf=DataCleaner(PercolatingClusters,Pointers,
                                                                      state,L,i,j)
            
            #clusters[0][k]==size of the cluster
            #clusters[1][k]==number of clusters with this size
            X=0
            Vol=0
            Vol_inf=0
            Vol2=0
            Vol2_inf=0
            Vol4=0
            Vol4_inf=0
            for k in range(len(clusters[0])-1):
                X+=clusters[0][k]*clusters[0][k]*clusters[1][k]
                aux=clusters[0][k]*clusters[1][k]
                Vol+=aux
                Vol2+=aux**2
                Vol4+=aux**4

            #Due to the sorting nature of np.unique the last entry in the array is reffered to Pointer=L,L
            #meaning a cluster of size 1:        
            X+=clusters[1][len(clusters[0])-1]
            aux=clusters[1][len(clusters[0])-1]
            Vol+=aux
            Vol2+=aux**2
            Vol4+=aux**4

            for k in range(len(clusters_inf[0])-1):
                aux=clusters_inf[0][k]*clusters_inf[1][k]
                Vol_inf+=aux
                Vol2_inf+=aux**2
                Vol4_inf+=aux**4
               
            aux=clusters_inf[1][len(clusters_inf[0])-1]
            Vol_inf+=aux
            Vol2_inf+=aux**2
            Vol4_inf+=aux**4
            
            P_infCluster=(Vol_inf-Vol)/Vol_inf
            P2_infCluster=(Vol2_inf-Vol2)/Vol2_inf
            P4_infCluster=(Vol4_inf-Vol4)/Vol4_inf
            
            if(P2_infCluster!=0):
                U=1.0-(1.0/3.0)*(P4_infCluster)/(P2_infCluster**2)
            else:
                U=1.0

            results[n,:]+=X/Vol,U,P_infCluster,P2_infCluster,P4_infCluster,Vol_inf

    
    for k in range(1,N-L,steps):  
        print(k,results[k,0]/Avg,results[k,1]/Avg,results[k,2]/Avg,results[k,3]/Avg,
              results[k,4]/Avg,results[k,5]/Avg,file=f)

        
    f.close()
        
        
#Matrix plot if needed:
#             print(n,state) 
#             print('\n')
#             print('Dark Blue (1) = Blocked Cells')
#             print('Different colours = different cluster')
#             sns.set_context('poster',font_scale=1)
#             sns.heatmap(Matrix,cmap='GnBu_r',annot=False)
            

    

In [22]:
%%time 
if __name__ == '__main__':
    main(40,500) 

CPU times: user 40min 15s, sys: 2.55 s, total: 40min 17s
Wall time: 40min 17s
